问答模型

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch import functional
from torch.utils.data import DataLoader, Dataset
# Load the tokenizer and model
from sacrebleu.metrics import BLEU

tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base")
model = T5ForConditionalGeneration.from_pretrained("Langboat/mengzi-t5-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

d:\ProgramData\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [17]:
print(device)
model

cuda


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

数据集处理

In [3]:
import json
class QADataSet(Dataset):
    def __init__(self, file_path):
        super().__init__()
        self.data_path = file_path
        self.data = self.load_data()
    
    def load_data(self):
        data = []
        with open(self.data_path, 'rt', encoding='utf-8') as f:
            for line in f:
                json_data = json.loads(line)
                question = json_data["question"]
                context = json_data["context"]
                data.append({
                    "input": f"问题是:{question},文章:{context}",
                    "answer": json_data["answer"]
                })
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    

In [8]:
from torch.utils.data import random_split
dataset = QADataSet('DuReaderQG/train.json')
train_dataset, valid_dataset = random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])
test_dataset = QADataSet('DuReaderQG/dev.json')
print(len(train_dataset), len(test_dataset))

11616 984


In [9]:
maxlength = 0
for data in train_dataset:
    inputs = tokenizer(data["input"],truncation=True, padding=True, max_length=10240, return_tensors="pt")
    maxlength = max(maxlength, inputs.input_ids.shape[1])
    inputs = tokenizer(data["answer"],truncation=True, padding=True, max_length=10240, return_tensors="pt")
    maxlength = max(maxlength, inputs.input_ids.shape[1])

for data in test_dataset:
    inputs = tokenizer(data["input"],truncation=True, padding=True, max_length=10240, return_tensors="pt")
    maxlength = max(maxlength, inputs.input_ids.shape[1])
    inputs = tokenizer(data["answer"],truncation=True, padding=True, max_length=10240, return_tensors="pt")
    maxlength = max(maxlength, inputs.input_ids.shape[1])
maxlength

1139

In [6]:
inputs = tokenizer(train_dataset[0]["input"], truncation=True, padding=True, max_length=1280, return_tensors="pt")
print(inputs)
print(tokenizer.convert_ids_to_tokens(inputs.input_ids[0]))

{'input_ids': tensor([[ 8080,    13,    43,   270,  3058,  1083,    56,  6664,  1261,    11,
           379,   645,   647,     3,  1385,    13,    43,   270,  3058,  1083,
            56,  6664,  1261,    22, 16871,  2117,   762,  2265,    25,     7,
          4889,  4155,   636,  6664,  1261,    22, 16871,  8952,    25,     7,
           555,  5704,  1261, 17776,    22, 16871,  5425,   762,   672,    94,
            25,  8362,     5, 16882,  2403,    22, 16871,   419,   108,   762,
           419,   173,    25,     7,  6187,     5, 13643,    22, 16871,  5360,
            94,   762,  5360,    99,    25,     7,  2503,  1441,  1643,  9522,
            22, 16871,   138,  3690,    25,     7,    42,  3610,   429,  8409,
            13,  8477,   346,     5, 10992,    22, 16871,   108,  1939, 13140,
          2630,    25,     7, 21792,    43,   270,  3058,  7839, 18788,  3792,
            22, 16871,   108,  1258, 13140,  7320,    25,     7,   126,  7907,
         15386,    22, 16871,   108,  

In [10]:
def collate_fn(batch):
    inputs = []
    answers = []
    for b in batch:
        inputs.append(b["input"])
        answers.append(b["answer"])

    batch_data = tokenizer(inputs, truncation=True, padding=True, max_length=1280, return_tensors="pt")

    with tokenizer.as_target_tokenizer():
        answer_token = tokenizer(answers, truncation=True, padding=True, max_length=1280, return_tensors="pt").input_ids
        
        batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(answer_token)
        eos_token_id = torch.where(answer_token == tokenizer.eos_token_id)[1]
        for idx, eos_id in enumerate(eos_token_id):
            answer_token[idx][eos_id + 1:] = -100  # Mask out the tokens after the EOS token
        batch_data['labels'] = answer_token
    
    return batch_data

In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [12]:
batch = next(iter(train_loader))
print(batch.keys())
print('batch shape:', {k: v.shape for k, v in batch.items()})
print(batch['decoder_input_ids'][0])
print(batch['labels'][0])

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'])
batch shape: {'input_ids': torch.Size([32, 824]), 'attention_mask': torch.Size([32, 824]), 'decoder_input_ids': torch.Size([32, 16]), 'labels': torch.Size([32, 16])}
tensor([   0, 1238, 2015,  280,    1,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
tensor([1238, 2015,  280,    1, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100])


d:\ProgramData\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


训练

In [ ]:
from tqdm.auto import tqdm
def train_loop(dataloader,model,optimizer,epoch, lr_scheduler,total_loss,device):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1) * len(dataloader)
    model.train()
    for data in dataloader:
        data = data.to(device)
        output = model(**data)
        loss = output.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss / (finish_batch_num + progress_bar.n):>7f}')
        progress_bar.update(1)
        


In [ ]:
# bleu1
blue_1 = BLEU(max_ngram_order=1)
blue_2 = BLEU(max_ngram_order=2)
blue_3 = BLEU(max_ngram_order=3)
blue_4 = BLEU()